In [1]:
!gdown --id 1n0j5zbMXMVnLPUqadFNcCxiZRu0ChoXL

Downloading...
From: https://drive.google.com/uc?id=1n0j5zbMXMVnLPUqadFNcCxiZRu0ChoXL
To: /content/Programming_Assignment_3.zip
42.0MB [00:00, 68.3MB/s]


In [2]:
!unzip "Programming_Assignment_3.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/train/pos/55_9.txt  
  inflating: Dataset/train/pos/550_10.txt  
  inflating: Dataset/train/pos/5500_10.txt  
  inflating: Dataset/train/pos/5501_10.txt  
  inflating: Dataset/train/pos/5502_10.txt  
  inflating: Dataset/train/pos/5503_10.txt  
  inflating: Dataset/train/pos/5504_10.txt  
  inflating: Dataset/train/pos/5505_10.txt  
  inflating: Dataset/train/pos/5506_10.txt  
  inflating: Dataset/train/pos/5507_7.txt  
  inflating: Dataset/train/pos/5508_10.txt  
  inflating: Dataset/train/pos/5509_10.txt  
  inflating: Dataset/train/pos/551_8.txt  
  inflating: Dataset/train/pos/5510_10.txt  
  inflating: Dataset/train/pos/5511_8.txt  
  inflating: Dataset/train/pos/5512_7.txt  
  inflating: Dataset/train/pos/5513_7.txt  
  inflating: Dataset/train/pos/5514_9.txt  
  inflating: Dataset/train/pos/5515_7.txt  
  inflating: Dataset/train/pos/5516_7.txt  
  inflating: Dataset/train/pos/5517_9.txt  
  inflating: Datas

In [3]:
import os
import numpy as np
from matplotlib import pyplot
%matplotlib inline
import pandas as pd
import re
import scipy
import itertools
from scipy.spatial.distance import cdist
from collections import Counter
from random import choice
import math
import glob
import itertools

In [4]:
#Formulating the train data-set using the text files.

tr_corpus1 = []

tr_files1 = glob.glob("Dataset/train/neg/*.txt")
for file_path in tr_files1:
    with open(file_path, 'r') as file_input:
        tr_corpus1.append(file_input.read())

tr_corpus2 = []

tr_files2 = glob.glob("Dataset/train/pos/*.txt")
for file_path in tr_files2:
    with open(file_path, 'r') as file_input:
        tr_corpus2.append(file_input.read())

tr_df1 = pd.DataFrame(tr_corpus1,columns=['Review'])
tr_df1['Label'] = 0
tr_df2 = pd.DataFrame(tr_corpus2,columns=['Review'])
tr_df2['Label'] = 1

train = pd.concat([tr_df1,tr_df2],ignore_index=True)

In [ ]:
#Formulating the test data-set using the text files.

ts_corpus1 = []

ts_files1 = glob.glob("Dataset/test/neg/*.txt")
for file_path in ts_files1:
    with open(file_path, 'r') as file_input:
        ts_corpus1.append(file_input.read())

ts_corpus2 = []

ts_files2 = glob.glob("Dataset/test/pos/*.txt")
for file_path in ts_files2:
    with open(file_path, 'r') as file_input:
        ts_corpus2.append(file_input.read())

ts_df1 = pd.DataFrame(ts_corpus1,columns=['Review'])
ts_df1['Label'] = 0
ts_df2 = pd.DataFrame(ts_corpus2,columns=['Review'])
ts_df2['Label'] = 1

test = pd.concat([ts_df1,ts_df2],ignore_index=True)

# **Preprocessing**

In [6]:
train['Review'] = train['Review'].str.lower() # Ensuring all words in the Tweet column of training data are lowercased
test['Review'] = test['Review'].str.lower() # Ensuring all words in the Tweet column of test data are lowercased

In [7]:
# Parsing the stop_words.txt file and storing all the words in a list.
stopwords = []
with open('Dataset/stop_words.txt','r') as file:    
    for line in file:         
        for word in line.split():            
            stopwords.append(word) 

In [8]:
# Removing all stopwords from all the reviews in training data.
train["Review"] = train["Review"].apply(lambda func: ' '.join(sw 
                                          for sw in func.split() 
                                          if sw not in stopwords))

# Removing all stopwords from all the reviews in test data.
test["Review"] = test["Review"].apply(lambda func: ' '.join(sw 
                                          for sw in func.split() 
                                          if sw not in stopwords))

In [9]:
#List of all special characters that are to be removed.
special_chars = ["!",'"',"%","&","amp","'","(",")", "*","+",",","-",".",
                 "/",":",";","<","=",">","?","[","\\","]","^","_",
                 "`","{","|","}","~","–","@","#","$"]

In [10]:
#Training Data
train['Review'] = train['Review'].str.replace(r'http?://[^\s<>"]+|www\.[^\s<>"]+', '') # Removing hyperlinks from all the reviews. They are not needed for classification.
train['Review'] = train['Review'].str.replace('@[A-Za-z0-9]+', '') # Removing usernames from all the reviews.
train['Review'] = train['Review'].str.replace(r'\B#\w*[a-zA-Z]+\w*', '') # Removing hashtags, including the text, from all the reviews. Hashtags are useless since their words cannot be splitted with spaces.
train['Review'] = train['Review'].str.replace('\d+', '') # Removing numbers from all the reviews. They will not assist in any way to improve the classification process.

for c in special_chars:
    train['Review'] = train['Review'].str.replace(c,'') # Removing all special characters from all the reviews

#Test Data
test['Review'] = test['Review'].str.replace(r'http?://[^\s<>"]+|www\.[^\s<>"]+', '') # Removing hyperlinks from all the reviews
test['Review'] = test['Review'].str.replace('@[A-Za-z0-9]+', '') # Removing usernames from all the reviews.
test['Review'] = test['Review'].str.replace(r'\B#\w*[a-zA-Z]+\w*', '') # Removing hashtags, including the text, from all the reviews
test['Review'] = test['Review'].str.replace('\d+', '') # Removing numbers from all the reviews

for c in special_chars:
    test['Review'] = test['Review'].str.replace(c,'') # Removing all special characters from all the reviews

#**Part 1**

In [11]:
C = train['Label'].unique()
loglikelihood = {0:{},1:{}} # Dictionary for loglikelihood with classes as keys, each having sub-dictionaries containing its words and their respective loglikelihoods.
logprior = list()
bigdoc = [[] for i in range(2)] # A list containing two lists, one each for class 0 and class 1

In [12]:
# All unique words in train['Reviews']
words = train['Review'].str.findall("\w+")
vocab = set()
for w in words:
    vocab.update(w)

# All words in the reviews that were deemed as negative
BD1 = list()
for value in train.loc[0:12499,'Review'].str.split():
    BD1.extend(value)
v1 = set(BD1)

# All words in the reviews that were deemed as positive
BD2 = list()
for value in train.loc[12500:24999,'Review'].str.split():
    BD2.extend(value)
v2 = set(BD2)

print(len(vocab),len(BD1),len(BD2))

117620 1556976 1608572


In [13]:
# Algorithm to calculate priors and likelihoods
for c in C:
  Ndoc = len(train['Label'])
  Nc = train.Label.value_counts()[0]
  logprior.append(math.log(Nc/Ndoc))
  V = list(vocab)

  if c == 0:
    bigdoc[c] = BD1
  if c == 1:
    bigdoc[c] = BD2
  
  bd_occurences = Counter(bigdoc[c])
  vocab_occurences = Counter(V)
  count_all_words = sum(vocab_occurences.values())
  
  for w in V:
    count_word_class = bd_occurences[w]
    loglikelihood[c][w] = (math.log((count_word_class+1)/(count_all_words+len(V)))) 


In [14]:
logprior

[-0.6931471805599453, -0.6931471805599453]

In [15]:
loglikelihood

{0: {'stjerner': -11.675214368013947,
  'troi': -12.368361548573892,
  'suckfest': -10.576602079345838,
  'concocting': -11.675214368013947,
  'plaintive': -12.368361548573892,
  'healing': -9.970466275775522,
  'irate': -10.982067187454001,
  'oskorblyonnye': -12.368361548573892,
  'formulascrappy': -12.368361548573892,
  'dumping': -9.970466275775522,
  'superlame': -11.269749259905783,
  'filmfriendly': -12.368361548573892,
  'brookmyres': -10.982067187454001,
  'severelooking': -11.675214368013947,
  'shore': -9.372629275019902,
  'blondieromantic': -11.675214368013947,
  'jesuss': -10.576602079345838,
  'tribbiani': -11.675214368013947,
  'serenitybr': -11.675214368013947,
  'gastaldi': -12.368361548573892,
  'pornlike': -11.675214368013947,
  'posthumously': -12.368361548573892,
  'belannas': -12.368361548573892,
  'casecheck': -12.368361548573892,
  'wiggleroom': -11.675214368013947,
  'paraphrased': -10.982067187454001,
  'mariana': -10.982067187454001,
  'tvstyled': -12.368361

In [16]:
sum = [0,0]
test['Predicted Label'] = ' '
newV = set(V) # Converting vocab into a set to ensure quicker computation

In [17]:
# Algorithm to allot predicted labels to each movie review
for indexer,row in test.iterrows():

  for c in C:
    sum[c] = logprior[c]
    sentence = list(test['Review'][indexer].split(" "))

    for i in sentence:
      word = i
      if word in newV:
        sum[c] = sum[c]+loglikelihood[c][word]
        
    argmax = sum.index(max(sum))
    test['Predicted Label'][indexer] = argmax

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
test

,Review,Label,Predicted Label
0,expecting born kill exciting hightension film ...,0,1
1,watched first episodes short back felt take an...,0,0
2,movie is likelihood worst film ever made certa...,0,0
3,hello normally love movies seen many dislike...,0,0
4,ravi chopra wrote film years back wanted cast...,0,0
...,...,...,...
24995,watched pp night say impressed real film seeme...,1,1
24996,favourite story tales crypt brion jones born p...,1,1
24997,the man knew much alfred hitchcocks remake t...,1,1
24998,film exactly remained fresh minds film buffs c...,1,1


In [19]:
acc = 0
for iter,row in test.iterrows():
  if row['Predicted Label'] == row['Label']:
    acc += 1

p1_accuracy = (acc/len(test['Review']))*100
print("Accuracy calculated manually: {}%".format(p1_accuracy))

Accuracy calculated manually: 83.28%


#**Part 2**

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [21]:
train_corpus = train['Review'].tolist()
test_corpus = test['Review'].tolist()

In [22]:
y_train = train['Label'].to_numpy()

In [23]:
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(train_corpus)
X_test = vectorizer.transform(test_corpus)

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_test = clf.predict(X_test)

In [24]:
accuracy = accuracy_score(y_train, y_test)*100
print("Accuracy calculated with sci-kit learn: {}%".format(accuracy))

Accuracy calculated with sci-kit learn: 82.568%


In [25]:
confusion_mat = confusion_matrix(y_train, y_test)
print(confusion_mat)

[[11004  1496]
 [ 2862  9638]]
